In [28]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import numpy as np
import os
import cv2

In [29]:
# Path ke folder dataset
data_folder = "DS-6"

In [30]:
# Ekstrak data dan label dari folder
def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            filepath = os.path.join(folder, filename)
            image = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # Baca sebagai grayscale
            if image is not None:
                image = cv2.resize(image, (28, 28))  # Resize ke 28x28
                images.append(image)
                label = filename.split('_')[0]
                labels.append(int(label) if label.isdigit() else 0)  # Default ke 0 jika label tidak valid
    return np.array(images), np.array(labels)

In [31]:
# Load dataset
print("Loading images...")
images, labels = load_images_from_folder(data_folder)
if len(images) == 0:
    print("No valid images found in the folder. Exiting...")
    exit()

print(f"Loaded {len(labels)} images.")

Loading images...
Loaded 8549 images.


In [32]:
# Normalisasi data
images = images / 255.0  # Normalisasi nilai piksel ke [0, 1]
images = images.reshape(-1, 28, 28, 1)  # Tambahkan channel untuk CNN
labels = to_categorical(labels, num_classes=10)  # One-hot encoding

In [33]:
# Split data untuk latih dan uji
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.4, random_state=42)

In [34]:
# Augmentasi data
datagen = ImageDataGenerator(rotation_range=10, width_shift_range=0.1, height_shift_range=0.1)
datagen.fit(X_train)

In [35]:
# Membangun model CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [36]:
# Kompilasi model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [37]:
# Pelatihan model
print("Training the model...")
history = model.fit(datagen.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_test, y_test),
                    epochs=20, verbose=2)

model.save('model/cnn_model.h5')


Training the model...
Epoch 1/20
161/161 - 4s - 22ms/step - accuracy: 0.1306 - loss: 2.2899 - val_accuracy: 0.1749 - val_loss: 2.1918
Epoch 2/20
161/161 - 2s - 13ms/step - accuracy: 0.3391 - loss: 1.8879 - val_accuracy: 0.7117 - val_loss: 1.0417
Epoch 3/20
161/161 - 2s - 12ms/step - accuracy: 0.5514 - loss: 1.3403 - val_accuracy: 0.8333 - val_loss: 0.6698
Epoch 4/20
161/161 - 2s - 11ms/step - accuracy: 0.6489 - loss: 1.0753 - val_accuracy: 0.8836 - val_loss: 0.4494
Epoch 5/20
161/161 - 2s - 11ms/step - accuracy: 0.6992 - loss: 0.9155 - val_accuracy: 0.8749 - val_loss: 0.4109
Epoch 6/20
161/161 - 2s - 12ms/step - accuracy: 0.7491 - loss: 0.8075 - val_accuracy: 0.9184 - val_loss: 0.2923
Epoch 7/20
161/161 - 2s - 12ms/step - accuracy: 0.7744 - loss: 0.7131 - val_accuracy: 0.9287 - val_loss: 0.2509
Epoch 8/20
161/161 - 2s - 11ms/step - accuracy: 0.7910 - loss: 0.6598 - val_accuracy: 0.9298 - val_loss: 0.2418
Epoch 9/20
161/161 - 2s - 11ms/step - accuracy: 0.8138 - loss: 0.5914 - val_accura

In [38]:
# Evaluasi model
print("Evaluating the model...")
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc * 100:.2f}%")


Evaluating the model...
107/107 - 0s - 2ms/step - accuracy: 0.9769 - loss: 0.1037
Test accuracy: 97.69%


In [39]:
# Prediksi
print("Making predictions...")
predictions = model.predict(X_test)
predicted_labels = np.argmax(predictions, axis=1)
true_labels = np.argmax(y_test, axis=1)

Making predictions...
107/107 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [40]:
# Menampilkan beberapa hasil
print("True labels vs Predicted labels:")
for i in range(10):
    print(f"True: {true_labels[i]}, Predicted: {predicted_labels[i]}")

True labels vs Predicted labels:
True: 3, Predicted: 3
True: 9, Predicted: 9
True: 5, Predicted: 5
True: 2, Predicted: 2
True: 8, Predicted: 8
True: 5, Predicted: 5
True: 4, Predicted: 4
True: 9, Predicted: 9
True: 6, Predicted: 6
True: 5, Predicted: 5
